## Segmenting and Clustering Neighborhoods of Toronto, Canada
In this notebook, I will be scraping Toronto's neighborhood data from wikipedia and use Foursquare API to find the most common venues around the Downtown Toronto area. 

Some sections of codes are taken from Coursera's IBM Applied Data Science Capstone assignments.

In [1]:
# Import and download all necessary libraries for this notebook
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup
from urllib.request import urlopen

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

! pip install folium==0.5.0
import folium

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Retrieve Toronto's neighborhood data
Web scrape data from wikipedia using python's BeautifulSoup library

In [2]:
# Open url to scrape table data from website
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [3]:
# Read the first table on the website
table = soup.find_all("table")[0]
print(table)

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

In [4]:
# Put scraped data into pandas dataframe
num_rows = 0
column_names = []

# count numbers of rows in table
for row in table.find_all('tr'):
    num_td = row.find_all('td')
    if len(num_td) > 0:
        num_rows += 1

# get headers of the table
num_th = table.find_all('th')
if len(num_th) > 0:
    for th in num_th:
        column_names.append(th.get_text(strip=True))
    
# create pandas dataframe with header names and include all rows in the table
df = pd.DataFrame(columns=column_names,index=range(0,num_rows))

row_position = 0
for row in table.find_all('tr'):
    column_position = 0
    columns = row.find_all('td')
    for column in columns:
        df.iat[row_position,column_position] = column.get_text(strip=True)
        column_position += 1
    if len(columns) > 0:
        row_position += 1

df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
# Remove rows in Borough column with 'Not assigned' value
df = df[df.Borough != 'Not assigned']
df.reset_index(drop=True, inplace=True)

print(df.shape)
df.head()

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
# Import geospatial data from the csv file
geospatial_df = pd.read_csv('http://cocl.us/Geospatial_data')
geospatial_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
# Merge the two dataframes on the postal code value to add the latitude and longitude to the dataframe
merged_df = pd.merge(df, geospatial_df, how='inner', left_on='Postal Code', right_on='Postal Code')
merged_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [8]:
# Find latitude and longitude of Toronto, Canada
address = "Toronto, Canada"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates for Toronto, Canada is {}, {}'.format(latitude, longitude))

Coordinates for Toronto, Canada is 43.6534817, -79.3839347


In [9]:
# Map all values of Toronto, Canada to a folium map
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=11)

for lat, lon, postal, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Postal Code'], merged_df['Borough'], merged_df['Neighbourhood']):
    label = '{}, {}, {}'.format(neighborhood, borough, postal)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lon],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='#3186cc',
                       parse_html=False).add_to(map_toronto)
    
map_toronto

## Examining Neighborhoods in Downtown Toronto
I will be examining the neighborhoods in Downtown Toronto as it is a dense area compared to other areas.

In [10]:
# Filter out Downtown Toronto from the merged_df dataframe
downtown_toronto = merged_df[merged_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_toronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


In [11]:
# Find latitude and longitude of Downtown Toronto, Canada
address = "Downtown Toronto, Canada"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates for Downtown Toronto, Canada is {}, {}'.format(latitude, longitude))

Coordinates for Downtown Toronto, Canada is 43.6563221, -79.3809161


In [12]:
# Map all points of Downtown Toronto on a folium map
map_downtown_toronto = folium.Map(location=[latitude,longitude], zoom_start=13)

for lat, lon, postal, borough, neighborhood in zip(downtown_toronto['Latitude'], downtown_toronto['Longitude'], downtown_toronto['Postal Code'], downtown_toronto['Borough'], downtown_toronto['Neighbourhood']):
    label = '{}, {}, {}'.format(neighborhood, borough, postal)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lon],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='#3186cc',
                       parse_html=False).add_to(map_downtown_toronto)
    
map_downtown_toronto

In [13]:
# The code was removed by Watson Studio for sharing.

In [14]:
# Connect to Foursquare account
# CLIENT_ID and CLIENT_SECRET are hidden
VERSION = '20180604' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [15]:
# Define function to retrieve nearby venues around Downtown Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto['Neighbourhood'],latitudes=downtown_toronto['Latitude'],longitudes=downtown_toronto['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [17]:
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(1241, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [18]:
print('Unique venues: {}'.format(len(downtown_toronto_venues['Venue Category'].unique())))

Unique venues: 210


In [19]:
# Create a new dataframe for venue category
downtown_toronto_dummy = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

downtown_toronto_dummy.insert(0, "Neighbourhood", downtown_toronto_venues['Neighborhood'])

print(downtown_toronto_dummy.shape)
downtown_toronto_dummy.head()

(1241, 211)


,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
downtown_toronto_grouped = downtown_toronto_dummy.groupby('Neighbourhood').mean().reset_index()
downtown_toronto_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0000,0.0000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,...,0.0000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00,0.00,0.015385,0.000000,0.015385,0.000000,0.015385,0.000000,0.015385
3,Christie,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.0125,0.0125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,...,0.0125,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000
5,"Commerce Court, Victoria Hotel",0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.0000,0.00,0.00,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
6,"First Canadian Place, Underground city",0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.0000,0.00,0.01,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
7,"Garden District, Ryerson",0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00,0.00,0.000000,0.010000,0.000000,0.010000,0.010000,0.000000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00,0.01,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00,0.00,0.061538,0.000000,0.000000,0.046154,0.015385,0.000000,0.000000


In [21]:
num_top_venues = 5

for neighborhood_name in downtown_toronto_grouped['Neighbourhood']:
    print("---"+neighborhood_name+"---")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighbourhood'] == neighborhood_name].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---Berczy Park---
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2            Beer Bar  0.03
3          Restaurant  0.03
4  Seafood Restaurant  0.03


---CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport---
              venue  freq
0  Airport Terminal  0.14
1    Airport Lounge  0.14
2   Airport Service  0.14
3     Boat or Ferry  0.07
4  Sculpture Garden  0.07


---Central Bay Street---
                       venue  freq
0                Coffee Shop  0.17
1             Sandwich Place  0.06
2                       Café  0.05
3         Italian Restaurant  0.05
4  Middle Eastern Restaurant  0.03


---Christie---
           venue  freq
0  Grocery Store  0.27
1           Café  0.20
2           Park  0.13
3     Restaurant  0.07
4    Candy Store  0.07


---Church and Wellesley---
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.06
2  Japanese Restaurant  0.06
3     

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighbourhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Farmers Market,Bakery,Cheese Shop,Pharmacy,Restaurant,Pub
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Rental Car Location,Sculpture Garden
2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burger Joint,Thai Restaurant,Salad Place,Bubble Tea Shop,Middle Eastern Restaurant,Park
3,Christie,Grocery Store,Café,Park,Coffee Shop,Nightclub,Candy Store,Italian Restaurant,Restaurant,Baby Store,Department Store
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Café,Men's Store,Mediterranean Restaurant,Hotel


In [24]:
# set number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 2, 1, 1, 1, 1, 1, 4], dtype=int32)

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_toronto_merged = downtown_toronto

# merge downtown_toronto_merged with downtown_toronto to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

downtown_toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Café,Park,Bakery,Breakfast Spot,Theater,Brewery,Electronics Store,Event Space
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Restaurant,Portuguese Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Bookstore,Pizza Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,American Restaurant,Clothing Store,Cocktail Bar,Cosmetics Shop,Gym,Park,Cheese Shop,Moroccan Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Farmers Market,Bakery,Cheese Shop,Pharmacy,Restaurant,Pub


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighbourhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1: Airports
Airport lounges, services and terminals are the most common venues in this area as it is next to an airport, Billy Bishop Toronto City Airport.
This cluster is indicated with a red dot in the above map.

In [27]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Rental Car Location,Sculpture Garden


## Cluster 2: Coffee Shops
Coffee shops, cafes and sushi restaurants are the most common venues in this area. This area is next to the harbor so it makes sense to have coffee shops where you can sit and enjoy the view and a nice cup of coffee as well as enjoy fresh caught seafood at the sushi restaurants. This area is indicated with a purple dot on the map.

In [28]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Pub,Café,Park,Bakery,Breakfast Spot,Theater,Brewery,Electronics Store,Event Space
1,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Restaurant,Portuguese Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
2,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Bookstore,Pizza Place
3,Downtown Toronto,1,Coffee Shop,Café,American Restaurant,Clothing Store,Cocktail Bar,Cosmetics Shop,Gym,Park,Cheese Shop,Moroccan Restaurant
4,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Farmers Market,Bakery,Cheese Shop,Pharmacy,Restaurant,Pub
5,Downtown Toronto,1,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burger Joint,Thai Restaurant,Salad Place,Bubble Tea Shop,Middle Eastern Restaurant,Park
7,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Gym,Thai Restaurant,Deli / Bodega,Clothing Store,Salad Place,Bookstore
8,Downtown Toronto,1,Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Brewery,Scenic Lookout,Restaurant,Fried Chicken Joint,History Museum
9,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,American Restaurant,Beer Bar
10,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Italian Restaurant,Seafood Restaurant,Deli / Bodega,Japanese Restaurant


## Cluster 3: Grocery Stores
Grocery stores are the most common venues in this area indicated with a blue dot in the map as this area is situated in the residential area.

In [29]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,2,Grocery Store,Café,Park,Coffee Shop,Nightclub,Candy Store,Italian Restaurant,Restaurant,Baby Store,Department Store


## Cluster 4: Parks
Parks, playgrounds and trails are the most common venues in this area indicated with a green dot on the map. This area is surrounded with nature so it makes sense that the most common venue is parks and trails so residents can enjoy some nature and get some daily exercise in at the trails.

In [30]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Park,Playground,Trail,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


## Cluster 5: Cafes
Cafes and bars are the most common venues in this area as it is near the University of Toronto. Students may study and work on their projects at cafes and head out to the bars after finishing exams to have fun and destress. This area is indicated with an orange dot on the map.

In [31]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Downtown Toronto,4,Café,Bar,Italian Restaurant,Japanese Restaurant,Bookstore,Bakery,Yoga Studio,Sandwich Place,Beer Bar,Beer Store
12,Downtown Toronto,4,Café,Vegetarian / Vegan Restaurant,Dessert Shop,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Comfort Food Restaurant,Bar,Bakery,Arts & Crafts Store
